In [ ]:
# Citibike daily trip count by station

from datetime import date, datetime, timedelta
import pandas as pd
import csv
import os

In [ ]:
rootdir = 'C:/Users/geoff/PycharmProjects/citibike/citibike_trip_data/citibike_all_months/'

In [ ]:
for file in os.listdir(rootdir):
    if file.endswith(".csv"):
        fh = open(rootdir + file)
        header = next(fh)

        # create a dated dictionary
        # with a nested dictionary to record trip starts and ends
        # for each station
        
        trip_dict = {}

        for line in fh:
            line = line.rstrip()
            line = line.split(',')
            date = datetime.strptime(line[1], '"%Y-%m-%d %H:%M:%S.%f"' ).date()
            start_station_id = str(line[4])
            end_station_id = str(line[8])

            if date not in trip_dict:
                trip_dict[date] = {}
            if start_station_id not in trip_dict[date]:
                trip_dict[date][start_station_id] = [1, 0]
            else:
                trip_dict[date][start_station_id][0] += 1
            if end_station_id not in trip_dict[date]:
                trip_dict[date][end_station_id] = [0, 1]
            else:
                trip_dict[date][end_station_id][1] += 1

        # create formatted list of daily trip counts for each station
        trip_list_formatted = []

        for trip_date in trip_dict.keys():
            for station in trip_dict[trip_date].keys():
                trip_list = [trip_date, station, trip_dict[trip_date][station][0], trip_dict[trip_date][station][1]]
                trip_list_formatted.append(trip_list)

        # sort the file
        trip_list_formatted.sort(key=lambda i: (i[0], i[1]))

        # create csv file for each month
        file_ext = file[:6]
        columns = ["date", "station_name", "trip_starts", "trip_ends"]
        df = pd.DataFrame(trip_list_formatted, columns=columns)
        df.to_csv("C:/Users/geoff/PycharmProjects/citibike/citibike_trip_data/citibike_summary_trip_counts/citibike_station_trip_counter_{}.csv".format(file_ext),
                  quoting=csv.QUOTE_NONE,
                  index=False)